# Cricbuzz API Data Fetching

This notebook demonstrates how to fetch cricket data from Cricbuzz API and push it to the database.

In [ ]:
import sys
sys.path.append('..')

import requests
import mysql.connector
from mysql.connector import Error
import os
from dotenv import load_dotenv
import pandas as pd

## Load Environment Variables

In [ ]:
load_dotenv('../.env')

# API Configuration
RAPIDAPI_KEY = "931ecde80cmsh80f2dc04d04d93ap136a0ejsnb76565e7402a"
RAPIDAPI_HOST = "cricbuzz-cricket.p.rapidapi.com"
BASE_URL = f"https://{RAPIDAPI_HOST}"

HEADERS = {
    "x-rapidapi-key": RAPIDAPI_KEY,
    "x-rapidapi-host": RAPIDAPI_HOST
}

## Connect to Database

In [ ]:
def get_db_connection():
    try:
        conn = mysql.connector.connect(
            host='localhost',
            user=os.getenv('DB_USER', 'root'),
            password=os.getenv('DB_PASSWORD', 'vicky@123'),
            database=os.getenv('DB_NAME', 'cricbuzz_db')
        )
        if conn.is_connected():
            print("✅ Database connected successfully")
            return conn
    except Error as e:
        print(f"❌ Database connection error: {e}")
        return None

conn = get_db_connection()

## Fetch Teams Data

In [ ]:
def fetch_teams():
    endpoint = f"{BASE_URL}/stats/v1/rankings/teams?formatType=test"
    response = requests.get(endpoint, headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Fetched teams data")
        return data
    else:
        print(f"❌ Error: {response.status_code}")
        return None

teams_data = fetch_teams()

## Fetch Recent Matches

In [ ]:
def fetch_recent_matches():
    endpoint = f"{BASE_URL}/matches/v1/recent"
    response = requests.get(endpoint, headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        print(f"✅ Fetched recent matches")
        return data
    else:
        print(f"❌ Error: {response.status_code}")
        return None

matches_data = fetch_recent_matches()

## Insert Data to Database

In [ ]:
# Example: Insert teams
if conn and teams_data:
    cursor = conn.cursor()
    
    # Insert logic here
    # cursor.execute("INSERT INTO teams ...")
    
    conn.commit()
    cursor.close()
    print("✅ Data inserted successfully")

## View Data

In [ ]:
# Query and display data
if conn:
    df = pd.read_sql("SELECT * FROM teams LIMIT 10", conn)
    display(df)

In [ ]:
# Close connection
if conn:
    conn.close()
    print("🔌 Database connection closed")